<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from collections import Counter

Import cleaned version of colexification dataframe

In [5]:
!gdown --id 1NB1f6mV51wf9P40SCO8WtouAAy6g1Ub4
df_colex = pd.read_csv('df_colexifications_new.csv')

Downloading...
From: https://drive.google.com/uc?id=1NB1f6mV51wf9P40SCO8WtouAAy6g1Ub4
To: /content/df_colexifications_new.csv
8.15MB [00:00, 71.6MB/s]


**knowing the data**: 183 linguistic fam

In [3]:
df_colex.groupby('Family').size()

Family
Abkhaz-Adyge     321
Abun               3
Afro-Asiatic    2736
Aikanã            21
Ainu              74
                ... 
Yuat               7
Yukaghir         177
Yámana           142
Zamucoan         191
Zuni             153
Length: 183, dtype: int64

Add concepticon_pair column for future purposes

In [6]:
df_colex['Concepticon_pair']=list(zip(df_colex['Concepticon_Gloss.x'],df_colex['Concepticon_Gloss.y']))

In [5]:
df_colex

,clics_form,Concepticon_ID.x,Glottocode,Concepticon_Gloss.x,Family,variety,Concepticon_ID.y,Concepticon_Gloss.y,Concepticon_pair
0,s@,1369,hrus1242,GOLD,Hruso,Hruso Aka Jamiri,1927,BAMBOO,"(GOLD, BAMBOO)"
1,s@,1369,hrus1242,GOLD,Hruso,Hruso Aka Jamiri,946,BLOOD,"(GOLD, BLOOD)"
2,avir@,1035,miji1239,GOOD,Sino-Tibetan,Dammai Dibin,923,LOVE,"(GOOD, LOVE)"
3,dzju,1425,hrus1242,GREEN,Hruso,Hruso Aka Jamiri,1424,YELLOW,"(GREEN, YELLOW)"
4,lah,1277,dakp1242,HAND,Sino-Tibetan,Monpa Changprong,639,MOUNTAIN,"(HAND, MOUNTAIN)"
...,...,...,...,...,...,...,...,...,...
132622,eni,763,gira1247,SKIN,Nuclear Trans New Guinea,girawa,1405,NAME,"(SKIN, NAME)"
132623,wus,763,payn1244,SKIN,Nuclear Trans New Guinea,paynamar,51,SORE,"(SKIN, SORE)"
132624,mping,2458,nend1239,PERSPIRE OR SWEAT,Nuclear Trans New Guinea,nend,1257,WING,"(PERSPIRE OR SWEAT, WING)"
132625,su,51,kein1239,SORE,Nuclear Trans New Guinea,kein,1257,WING,"(SORE, WING)"


In [7]:
# df_fam --> df with families and varieties
df_fam = df_colex[['Family','Glottocode']].copy()

In [8]:
df_fam=df_fam.drop_duplicates()


In [8]:
df_fam.groupby(['Family']).size()

Family
Abkhaz-Adyge     5
Abun             2
Afro-Asiatic    61
Aikanã           1
Ainu             1
                ..
Yuat             1
Yukaghir         2
Yámana           1
Zamucoan         1
Zuni             1
Length: 183, dtype: int64

In [9]:
dicFam = df_fam.groupby(['Family']).size().to_dict() # dictionary with number of variety for future purposes

**BOOTSTRAP**

use the df_fam as support to choose the varieties

In [10]:
new_df_fam = df_fam.groupby(['Family']).apply(lambda x: x.sample(n=(len(x['Glottocode'])), replace=True)).reset_index(drop = True)

In [11]:
new_df_fam.Family.value_counts() # check

Austronesian                339
Nuclear Trans New Guinea    283
Sino-Tibetan                177
Pama-Nyungan                165
Atlantic-Congo              133
                           ... 
Seri                          1
Movima                        1
Tsimshian                     1
Yuat                          1
Aymaran                       1
Name: Family, Length: 183, dtype: int64

In [12]:
# version 1
#new_df_fam = df_fam.groupby(['Family']).apply(lambda x: x.sample(n=2, replace=True)).reset_index(drop = True)
# take n=2 rows for every family
#new_df_fam.Family.value_counts() # check

In [13]:
# create list with varieties chosen
listVar=new_df_fam.Glottocode.tolist()

In [14]:
boot_df = pd.DataFrame()

for v in listVar:
  # for every variety, take all colexification and put them into boot_df
  boot_df=boot_df.append(df_colex.loc[df_colex['Glottocode'] == v], ignore_index = True)


In [15]:
boot_df

,clics_form,Concepticon_ID.x,Glottocode,Concepticon_Gloss.x,Family,variety,Concepticon_ID.y,Concepticon_Gloss.y,Concepticon_pair
0,chw@,1008,abaz1241,BULL,Abkhaz-Adyge,Abaza,1169,OX,"(BULL, OX)"
1,hwa,1337,abaz1241,PIG,Abkhaz-Adyge,Abaza,3154,WILD BOAR,"(PIG, WILD BOAR)"
2,wasa,1344,abaz1241,RAM,Abkhaz-Adyge,Abaza,1331,SHEEP,"(RAM, SHEEP)"
3,xr@x@ch,977,abaz1241,ARROW,Abkhaz-Adyge,Abaza,994,BOW,"(ARROW, BOW)"
4,kwaga,677,abaz1241,AXE,Abkhaz-Adyge,Abaza,386,BATTLE-AXE,"(AXE, BATTLE-AXE)"
...,...,...,...,...,...,...,...,...,...
134458,kum,281,zuni1245,POST,Zuni,Zuni,344,TREE TRUNK,"(POST, TREE TRUNK)"
134459,hele,1176,zuni1245,SPADE,Zuni,Zuni,1901,SHOVEL,"(SPADE, SHOVEL)"
134460,koyaci,337,zuni1245,PITCHFORK,Zuni,Zuni,409,FORKED BRANCH,"(PITCHFORK, FORKED BRANCH)"
134461,asa,2575,zuni1245,DO OR MAKE,Zuni,Zuni,1840,BUILD,"(DO OR MAKE, BUILD)"


In [16]:
#df_mean = pd.DataFrame() # dataframe with family and mean per family
#df_sum = pd.DataFrame(columns=['fam', 'col', 'rep'])
suppDf = []

def concept_sum(x):
  # x = df divided by concepticon inside only one family

  #totLang = len(set(x['variety'])) # how many languages have that colexification --> SET
  totLang = len(x['Glottocode']) # how many languages have that colexification faking to have different families from bootstrap --> NO SET
  # NB we don't have repetitions of Concepticon_pair in a variety, if we have it, it's just because we did the bootstrap with replacement
  
  newEl = [x['Family'].iloc[0], x['Concepticon_pair'].iloc[0], totLang]
  suppDf.append(newEl)

def colex_rep(x):
  # x = df divided by family
  x.groupby('Concepticon_pair').apply(lambda y: concept_sum(y))

In [17]:
# 1: how many times COL1 repetes in FAM1, FAM2...
# 2: sum of all the times for COL1
# 3: divide the sum by the tot FAMs

In [18]:
# not very clear: 
# n of languages that shows that colexification divided by total number of languages --> percentage
# mean per family: sum of number of languages that shows that colexification divided by total number of colexifications --> x languages per colexification

In [19]:
boot_df.groupby(['Family']).apply(lambda x: colex_rep(x))

""


In [20]:

df_sum = pd.DataFrame(suppDf, columns=['Family', 'Colex_pair', 'Var_rep'])

In [21]:
df_sum[df_sum['Var_rep']>1] # check

,Family,Colex_pair,Var_rep
4,Abkhaz-Adyge,"(ARROW, BOW)",3
5,Abkhaz-Adyge,"(AXE, BATTLE-AXE)",4
26,Abkhaz-Adyge,"(BRANCH, TWIG)",2
30,Abkhaz-Adyge,"(BULL, OX)",3
39,Abkhaz-Adyge,"(COME, ARRIVE)",2
...,...,...,...
62185,Zamucoan,"(YOUNG, UNRIPE)",2
62231,Zuni,"(GRANDMOTHER, GRANDDAUGHTER)",2
62237,Zuni,"(HOUSE, ROOM)",2
62238,Zuni,"(HOW, WHAT)",2


segunda parte: colexification frequency within each family

In [22]:
#df_sum['mean']=df_sum['rep'].apply(lambda x: x/2) # divided by 2 because we took only 2 varieties --> change if necessary

In [23]:
df_sum['Num_var']='' # new empty column

In [24]:
for i in range(len(df_sum['Colex_pair'])):
  fam = df_sum['Family'].iloc[i] # take the family for the row
  rep = df_sum['Var_rep'].iloc[i] # take var rep
  div = rep / dicFam[fam]
  df_sum['Num_var'].iloc[i] = div

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [25]:
df_sum

,Family,Colex_pair,Var_rep,Num_var
0,Abkhaz-Adyge,"(ALL, FULL)",1,0.2
1,Abkhaz-Adyge,"(APPLE, BARK)",1,0.2
2,Abkhaz-Adyge,"(APPLE, BEESWAX)",1,0.2
3,Abkhaz-Adyge,"(APPLE, LEATHER)",1,0.2
4,Abkhaz-Adyge,"(ARROW, BOW)",3,0.6
...,...,...,...,...
62330,Zuni,"(WIND, BREATH OR BREATHE)",1,1
62331,Zuni,"(WING, SILVER)",1,1
62332,Zuni,"(WOOD, FIREWOOD)",1,1
62333,Zuni,"(YES, YOUNG WOMAN)",1,1


the next step would be to sum over them

In [26]:
suppDf = []

def colex_freq(x):
  # x = df of only one colex_pair
  sumNumVar = sum(x['Num_var'])
  div = sumNumVar / 183
  newEl = [x['Colex_pair'].iloc[0], div]
  suppDf.append(newEl)

In [27]:
df_sum.groupby(['Colex_pair']).apply(lambda x: colex_freq(x))

""


In [28]:
df_colex_freq = pd.DataFrame(suppDf, columns=['Colex_pair', 'Freq'])

In [29]:
df_colex_freq

,Colex_pair,Freq
0,"(A LITTLE, WHAT)",0.000042
1,"(ABOVE, BELOW OR UNDER)",0.003643
2,"(ABOVE, BLACK)",0.000039
3,"(ABOVE, CART)",0.002732
4,"(ABOVE, CIRCLE)",0.002732
...,...,...
45775,"(YOUNGER SISTER, SELL)",0.002732
45776,"(YOUNGER SISTER, SIBLING)",0.003328
45777,"(YOUNGER SISTER, STEPDAUGHTER)",0.000228
45778,"(YOUNGER SISTER, YOUNGER BROTHER)",0.011027


TO DO 

create SOMETHING to repeat bootstrap 3 (x) veces

### BOOTSTRAP function

In [15]:
import time

def colex_freq(x, suppDf):
  # x = df of only one colex_pair
  sumNumVar = sum(x['Num_var'])
  div = sumNumVar / 183
  newEl = [x['Colex_pair'].iloc[0], div]
  suppDf.append(newEl)


def concept_sum(x, suppDf):
  # x = df divided by concepticon inside only one family
  totLang = len(x['Glottocode']) # how many languages have that colexification faking to have different families from bootstrap --> NO SET
  # NB we don't have repetitions of Concepticon_pair in a variety, if we have it, it's just because we did the bootstrap with replacement
  newEl = [x['Family'].iloc[0], x['Concepticon_pair'].iloc[0], totLang]
  suppDf.append(newEl)


def colex_rep(x, suppDf):
  # x = df divided by family
  x.groupby('Concepticon_pair').apply(lambda y: concept_sum(y, suppDf))


def bootstrapping(cicle):
  print ("Running bootstrapping function")
  start_time = time.time()

  listBoot = []
  # cicle = number of repetitions
  for k in range (cicle):
    # create new df with random varieties
    new_df_fam = df_fam.groupby(['Family']).apply(lambda x: x.sample(n=(len(x['Glottocode'])), replace=True)).reset_index(drop = True)
    # create list with varieties chosen
    listVar=new_df_fam.Glottocode.tolist()
    print("List of linguistic varieties chosen")
    # new df
    boot_df = pd.DataFrame()

    for v in listVar:
      # for every variety, take all colexification and put them into boot_df
      boot_df=boot_df.append(df_colex.loc[df_colex['Glottocode'] == v], ignore_index = True)

    suppDf = [] # new empty list

    print("Taking families grouped by concepticon pair and counting number of varieties with the same colexification")

    boot_df.groupby(['Family']).apply(lambda x: colex_rep(x, suppDf))
    print("Created dataframe with colexification from chosen varieties")
    # new df from list obtained by functions
    df_sum = pd.DataFrame(suppDf, columns=['Family', 'Colex_pair', 'Var_rep'])

    df_sum['Num_var']='' # new empty column

    for i in range(len(df_sum['Colex_pair'])):
      fam = df_sum['Family'].iloc[i] # take the family for the row
      rep = df_sum['Var_rep'].iloc[i] # take var rep
      div = rep / dicFam[fam]
      df_sum['Num_var'].iloc[i] = div

    suppDf = [] # empty the list

    print("Counting colex frequency among all families")
    df_sum.groupby(['Colex_pair']).apply(lambda x: colex_freq(x, suppDf))

    listBoot.append(suppDf) # add list to list of bootstrap
    # df_colex_freq = pd.DataFrame(suppDf, columns=['Colex_pair', 'Freq']) # to save in df
    print("Finished cicle number", k)
  print("--- %s seconds ---" % (time.time() - start_time))
  return listBoot


In [23]:
lBoot = bootstrapping(3)

Running bootstrapping function
List of linguistic varieties chosen
Taking families grouped by concepticon pair and counting number of varieties with the same colexification
Created dataframe with colexification from chosen varieties


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Counting colex frequency among all families
Finished cicle number 0
List of linguistic varieties chosen
Taking families grouped by concepticon pair and counting number of varieties with the same colexification
Created dataframe with colexification from chosen varieties
Counting colex frequency among all families
Finished cicle number 1
List of linguistic varieties chosen
Taking families grouped by concepticon pair and counting number of varieties with the same colexification
Created dataframe with colexification from chosen varieties
Counting colex frequency among all families
Finished cicle number 2
--- 285.95579767227173 seconds ---


In [24]:
len(lBoot)

3

In [25]:
df_colex_freq = pd.DataFrame(lBoot[2], columns=['Colex_pair', 'Freq']) # to save in df


In [26]:
df_colex_freq

,Colex_pair,Freq
0,"(A LITTLE, WHAT)",0.000042
1,"(ABOVE, BELOW OR UNDER)",0.005464
2,"(ABOVE, BLACK)",0.000151
3,"(ABOVE, CART)",0.005464
4,"(ABOVE, CIRCLE)",0.005464
...,...,...
40561,"(YOUNGER SISTER, SON)",0.000031
40562,"(YOUNGER SISTER, STEPDAUGHTER)",0.000455
40563,"(YOUNGER SISTER, THEN)",0.000031
40564,"(YOUNGER SISTER, YOUNGER BROTHER)",0.011142
